In [57]:
from PIL import Image
import numpy as np
from mylibdir.class_ica import ICA

#合成画像を作成
im1=np.array(Image.open('report_kadai4_data1/original1.png'))
im2=np.array(Image.open('report_kadai4_data1/original2.png'))
mix1=im1*0.4+im2*0.6
mix2=im1*0.6+im2*0.4
mix1=mix1.astype(np.uint8)
mix2=mix2.astype(np.uint8)

result_mix1=Image.fromarray(mix1)
result_mix1.save('report_kadai4_data1/image1.png')

result_mix2=Image.fromarray(mix2)
result_mix2.save('report_kadai4_data1/image2.png')

In [87]:
#画像の読み込み
im1=np.array(Image.open('report_kadai4_data1/image1.png'))
h1,w1,c1=im1.shape
data1_R=np.empty(h1*w1)
data1_G=np.empty(h1*w1)
data1_B=np.empty(h1*w1)

im2=np.array(Image.open('report_kadai4_data1/image2.png'))
h2,w2,c2=im2.shape
data2_R=np.empty(h2*w2)
data2_G=np.empty(h2*w2)
data2_B=np.empty(h2*w2)

if h1!=h2 or w1!=w2 or c1!=c2:
    raise ValueError('Format_Error')
else:
    h=h1
    w=w1
    nc=c1
    
n=0
for i in range(h):#ICA用に一次元配列に変換
    for j in range(w):
        data1_R[n]=im1[i,j,0]
        data1_G[n]=im1[i,j,1]
        data1_B[n]=im1[i,j,2]
        
        data2_R[n]=im2[i,j,0]
        data2_G[n]=im2[i,j,1]
        data2_B[n]=im2[i,j,2]
        n+=1

In [88]:
#各色についてICAを実行
data_R=[data1_R,data2_R]
data_G=[data1_G,data2_G]
data_B=[data1_B,data2_B]

y_R=ICA(data_R).run() # ICAの実行
y_R=[((y_R_i-min(y_R_i))/(max(y_R_i)-min(y_R_i))*256).astype(np.uint8) for y_R_i in np.asarray(y_R)]#符号なし8ビット整数型に変換

y_G=ICA(data_G).run() # ICAの実行
y_G=[((y_G_i-min(y_G_i))/(max(y_G_i)-min(y_G_i))*256).astype(np.uint8) for y_G_i in np.asarray(y_G)]#符号なし8ビット整数型に変換

y_B=ICA(data_B).run() # ICAの実行
y_B=[((y_B_i-min(y_B_i))/(max(y_B_i)-min(y_B_i))*256).astype(np.uint8) for y_B_i in np.asarray(y_B)]#符号なし8ビット整数型に変換

In [89]:
if np.absolute(y_R[0]-y_G[0]).mean()<np.absolute(y_R[0]-y_G[1]).mean():#各ピクセルのRGB値の差の平均値を比較し、同一画像の単色成分を判定
    a=0
    b=1
else:
    a=1
    b=0
    
if np.absolute(y_R[0]-y_B[0]).mean()<np.absolute(y_R[0]-y_B[1]).mean():
    c=0
    d=1
else:
    c=1
    d=0

result1=np.empty((h,w,nc)).astype(np.uint8)
result2=np.empty((h,w,nc)).astype(np.uint8)
n=0
for i in range(h):#三次元配列にもどす
    for j in range(w):
        result1[i,j,0]=y_R[0][n]
        result1[i,j,1]=y_G[a][n]
        result1[i,j,2]=y_B[c][n]
        
        result2[i,j,0]=y_R[1][n]
        result2[i,j,1]=y_G[b][n]
        result2[i,j,2]=y_B[d][n]
        n+=1
        
result_im1=Image.fromarray(result1)
result_im1.save('report_kadai4_data1/result1.png')

result_im2=Image.fromarray(result2)
result_im2.save('report_kadai4_data1/result2.png')